# Jacobi Method

Jacobi method is one of the stationary iterative method for solving a linear system $$A{\bf x}={\bf b}$$ It splits the matrix $A=D+U+L$, where $D$ represents the diagonal of the matrix $A$, and $U$ and $L$ represent the upper and lower triangular parts of the matrix $A$, respectively. Let ${\bf x}^{(k)}$ denotes the $k$-th iteration of the approximated solution, then the Jacobi method reads $${\bf x}^{(k+1)}=D^{-1}\left({\bf b}-(L+U){\bf x}^{(k)}\right)$$

Or componentwisely, it can be written as

$$x_i^{(k+1)}=\frac{1}{a_{ii}}\left(b_i-\sum_{j=0, j\neq i}^{n}a_{ij}x_j^{(k)}\right)$$

* Now, let's try to use Jacobi Method to solve a one dimentional Poissons equation:
$$-u_{xx}=\pi^2\sin{\pi x} \quad\text{ for } x\in(0,1)$$
$$\text{with}\quad u(0)=u(1)=0$$


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.sparse as sp
import numpy as np


In [2]:
def Poisson1D_Jacobi_OneStep(b, u, stencil = [-1, 2, -1]):
    v = np.copy(u)
    n = u.shape[0]
    for i in range(1, n-1):
        v[i] = (b[i]-stencil[0]*u[i+1]-stencil[2]*u[i-1])/stencil[1]
    return v

def Poisson1D_Jacobi(f, n, domain=[0,1], bdry_cond=[0,0], num_iter=1000, stencil=[-1, 2, -1]):
    # set up the numerical parameters, as well as initial step. 
    x = np.linspace(domain[0],domain[1],n+1)
    h = (domain[1]-domain[0])/n
    b = f(x)*h**2
    u = np.zeros(n+1)
    u[0] = bdry_cond[0]
    u[-1] = bdry_cond[1]
    # for loop or while loop for the iterations. 
    for i in range(num_iter):
        u = Poisson1D_Jacobi_OneStep(b, u, stencil=stencil)
    return u


In [3]:
f1 = lambda x: np.pi**2*np.sin(np.pi*x)
n1 = 100
u = Poisson1D_Jacobi(f1, n1, num_iter=10000)
x = np.linspace(0,1,n1+1)
plt.plot(x, u)

Let's use the code above to solve another problem: $$-u_{xx}=x(x-1)\cos(x)\quad\text{ for } x\in (0,\pi) $$
$$\text{ with }\quad u(0) = 1 \text{ and } u(\pi) = 2$$

In [4]:
f2 = lambda x: x*(x-1)*np.cos(x)
n2 = 100
u = Poisson1D_Jacobi(f2, n2, domain=[0, np.pi], bdry_cond=[1,2], num_iter=10000)
x = np.linspace(0,1,n1+1)
plt.plot(x, u)

Let's use the code above to solve another problem: $$u-u_{xx}=x(x-1)\cos(x)\quad\text{ for } x\in (0,\pi) $$
$$\text{ with }\quad u(0) = 1 \text{ and } u(\pi) = 2$$

In [5]:
f2 = lambda x: x*(x-1)*np.cos(x)
n2 = 100
u = Poisson1D_Jacobi(f2, n2, domain=[0, np.pi], bdry_cond=[1,2], num_iter=10000, stencil=[-1,2+(np.pi/n2)**2,-1])
x = np.linspace(0,1,n1+1)
plt.plot(x, u)

* Alright. It seems that you are getting the hang of Jacobi Method now. Let's try to use Jacobi Method to solve a $2D$ Poissons equation. 
$$-u_{xx}-u_{yy}=2\pi^2\sin{\pi x}\cos{\pi y} \quad\text{ for } x\in(0,1)\text{ and } y\in(0,1)$$
$$\text{with}\quad u(0,y)=u(1,y)=0$$
$$\text{and }\quad u(x,0)=\sin{\pi x}\quad u(x,1)=-\sin{\pi x}$$

In [2]:
def Poisson2D_Jacobi_onestep(b,u):
    n = u.shape[0]
    v = np.copy(u)
    for i in range(1,n-1):
        for j in range(1,n-1):
            v[i,j]=(b[i,j]+u[i+1,j]+u[i-1,j]+u[i,j-1]+u[i,j+1])/4
    return v

def Poisson2D_Jacobi(f, n, domain=[0, 1], bdry_cond=[[lambda x:0, lambda x:0], [lambda x:0, lambda x:0]], num_iter=1000):
    # bdry_cond = [[g1, g2],[g3, g4]], 
    # where u(0,y)=u[:,0]=g1(y), u(1,y)=u[:,n]=g2(y),
    # and   u(x,0)=u[0,:]=g3(x), u(x,1)=u[n,:]=g4(x)
    h = (domain[1]-domain[0])/n
    x = np.linspace(domain[0], domain[1], n+1)
    y = np.linspace(domain[0], domain[1], n+1)
    b = f(x,y)*h*h
    u = np.zeros((n+1,n+1))
    u[:,0] = bdry_cond[0][0](y)
    u[:,n] = bdry_cond[0][1](y)
    u[0,:] = bdry_cond[1][0](x)
    u[n,:] = bdry_cond[1][1](x)
    for k in range(num_iter):
        u = Poisson2D_Jacobi_onestep(b,u)
    return u

In [3]:
f = lambda x,y: np.pi**2*np.outer(np.cos(np.pi*y),np.sin(np.pi*x))
g1 = lambda y: 0
g2 = lambda y: 0
g3 = lambda x: np.sin(np.pi*x)
g4 = lambda x: -np.sin(np.pi*x)
n = 64
u = Poisson2D_Jacobi(f, n, bdry_cond=[[g1,g2],[g3,g4]], num_iter=10000)
fig = plt.figure(figsize =(14, 9))
ax = plt.axes(projection ='3d')
x = np.outer(np.linspace(0,1,n+1), np.ones(n+1))
y = x.copy().T
ax.plot_surface(x,y,u)
ax.view_init(10, 100)